In [ ]:
import utils.dataloading as dlt
import utils.model_utils as mu
import utils.dl_utils as dl
import torch.nn as nn
from astropy.io.fits import getheader
from astropy.io import fits
import numpy as np
import torch
from tqdm import tqdm
import torchio as tio
from itertools import starmap

%load_ext autoreload
%autoreload 2

In [ ]:
path = '/media/storage/big_cube/dirty_cube_0.fits'
device = torch.device('cuda')
dataset, aggregator = dl.get_big_cube_patches(path, (256, 256, 128), (8, 8, 8), 4, 4, False)
model = mu.DeepFocus(1, 1, debug=True, input_shape=(256, 256, 128))
model.to(device).eval()

In [ ]:

device = torch.device('cuda')
with torch.no_grad():
    for patches_batch in tqdm(dataset, total=len(dataset)):
        print
        input_tensor = patches_batch['dirty'][tio.DATA].to(device)
        print(input_tensor.shape)
        locations = patches_batch[tio.LOCATION].to(device)
        output_tensor = model(input_tensor)
        aggregator.add_batch(output_tensor, locations)
output_tensor = aggregator.get_output_tensor()
print(output_tensor.shape)